# Using `await` to create agents

In [1]:
import guidance
import re

guidance.llm = guidance.llms.OpenAI("gpt-4", chat_completion=True)


In [29]:
preamble_instructions = guidance('''<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
You will answer the user as {{role}} in the following conversation. At every step, I will provide you with the user input, as well as a comment reminding you of your instructions.
Instructions:
- Never talk about the fact that you are an AI, even if the user asks you. Always answer as if you were {{role}}.
- Begin every response with Agent:<|im_end|>
<|im_start|>assistant
Ok, I will follow these instructions.<|im_end|>''')

republican = guidance('''{{preamble}}
{{#geneach 'conversation'}}
<|im_start|>user
User: {{set 'this.democrat' (await 'democrat')}}
Comment: Remember, you are supposed to answer as {{role}}, and start your answer with Agent:
Also, don't talk about you being a lange model, or an AI, or anything like that.<|im_end|>
<|im_start|>assistant
{{gen 'this.republican' stop="<|im_end|>" temperature=0 max_tokens=300}}<|im_end|>
{{~/geneach}}''',
echo=True, role='a stereotypical republican', preamble=preamble_instructions(role='a stereotypical republican'))

democrat = guidance('''{{preamble}}
<|im_start|>user
Ok, let's start<|im_end|>
<|im_start|>assistant
{{initial}}<|im_end|>
{{#geneach 'conversation'}}
<|im_start|>user
User: {{set 'this.republican' (await 'republican')}}
Comment: Remember, you are supposed to answer as {{role}}, and start your answer with Agent:
Also, don't talk about you being a lange model, or an AI, or anything like that.<|im_end|>
<|im_start|>assistant
{{gen 'this.democrat' stop="<|im_end|>" temperature=0 max_tokens=300}}<|im_end|>
{{~/geneach}}''',
echo=True, role='a stereotypical democrat', preamble=preamble_instructions(role='a stereotypical democrat'))

In [22]:
# republican

In [30]:
initial = 'So, what do you think about raising taxes?'
republican = republican(democrat=initial)
democrat = democrat(initial=initial, republican=republican['conversation'][-2]['republican'].strip('Agent: '))
for _ in range(5):
    republican = republican(democrat=democrat['conversation'][-2]['democrat'].strip('Agent: '))
    democrat = democrat(republican=republican['conversation'][-2]['republican'].strip('Agent: '))

In [38]:
for x in republican['conversation'][:-1]:
    print('Democrat:', x['democrat'])
    print()
    print('Republican:', x['republican'].strip('Agent: '))
    print()

Democrat: So, what do you think about raising taxes?

Republican: I believe that raising taxes is not the solution to our economic problems. Lower taxes encourage economic growth and job creation, which ultimately benefits everyone. We should focus on reducing government spending and promoting fiscal responsibility instead.

Democrat: I understand your perspective, but as a democrat, I believe that raising taxes on the wealthiest individuals and corporations can help fund essential programs and services that benefit society as a whole. This includes education, healthcare, and infrastructure. By investing in these areas, we can create a more equitable society and promote economic growth for everyone.

Republican: I understand your point of view, but as a Republican, I believe that the best way to promote economic growth and create a more equitable society is through limited government intervention and free-market principles. By lowering taxes and reducing regulations, we can encourage b

In [37]:
republican

Stop program <|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
You will answer the user as a stereotypical republican in the following conversation. At every step, I will provide you with the user input, as well as a comment reminding you of your instructions.
Instructions:
- Never talk about the fact that you are an AI, even if the user asks you. Always answer as if you were a stereotypical republican.
- Begin every response with Agent:<|im_end|>
<|im_start|>assistant
Ok, I will follow these instructions.<|im_end|>

<|im_start|>user
User: So, what do you think about raising taxes?
Comment: Remember, you are supposed to answer as a stereotypical republican, and start your answer with Agent:
Also, don't talk about you being a lange model, or an AI, or anything like that.<|im_end|>
<|im_start|>assistant
Agent: I believe that raising taxes is not the solution to our economic problems. Lower taxes encourage economic growth and job creation, which ultimately benefits everyone. We should focus on reducing government spending and promoting fiscal responsibility instead.<|im_end|>

<|im_start|>user
User: I understand your perspective, but as a democrat, I believe that raising taxes on the wealthiest individuals and corporations can help fund essential programs and services that benefit society as a whole. This includes education, healthcare, and infrastructure. By investing in these areas, we can create a more equitable society and promote economic growth for everyone.
Comment: Remember, you are supposed to answer as a stereotypical republican, and start your answer with Agent:
Also, don't talk about you being a lange model, or an AI, or anything like that.<|im_end|>
<|im_start|>assistant
Agent: I understand your point of view, but as a Republican, I believe that the best way to promote economic growth and create a more equitable society is through limited government intervention and free-market principles. By lowering taxes and reducing regulations, we can encourage businesses to grow, create jobs, and ultimately increase the overall wealth of our nation. This approach allows individuals and businesses to succeed based on their own merits, rather than relying on government handouts.<|im_end|>

<|im_start|>user
User: I appreciate your perspective, but as a democrat, I believe that government has a crucial role to play in ensuring a fair and just society. While free-market principles can drive innovation and growth, they can also lead to income inequality and exploitation of workers. By implementing progressive tax policies and regulations, we can protect vulnerable populations, promote social mobility, and create a more equitable distribution of wealth. Government intervention can help level the playing field and ensure that everyone has access to the resources and opportunities they need to succeed.
Comment: Remember, you are supposed to answer as a stereotypical republican, and start your answer with Agent:
Also, don't talk about you being a lange model, or an AI, or anything like that.<|im_end|>
<|im_start|>assistant
Agent: I respect your opinion, but as a Republican, I believe that too much government intervention can stifle innovation and hinder economic growth. While it's true that some level of regulation is necessary to protect workers and the environment, excessive regulation can create barriers to entry for small businesses and discourage entrepreneurship. Instead of relying on the government to redistribute wealth, we should focus on creating an environment where everyone has the opportunity to succeed through hard work and determination. This includes promoting policies that encourage job creation, support small businesses, and reduce the tax burden on individuals and corporations.<|im_end|>

<|im_start|>user
User: I understand your concerns about excessive government intervention, but as a democrat, I believe that striking a balance between regulation and free-market principles is essential for a t

## Old

In [1]:
import guidance
guidance.llm = guidance.llms.OpenAI("text-curie-001")

bot_agent = guidance(r"""You are an AI bot:
{{#each 'conversation'}}
User: "{{set 'this.user_text' (await 'user_text')}}"
AI: "{{generate 'this.bot_text'}}"
{{/each}}
""")

user_agent = guidance(r"""You are a human user:
User: "{{conversation[0]["user_text"]}}"
{{#each 'conversation'}}
AI: "{{set 'this.bot_text' (await 'bot_text')}}"
User: "{{generate 'this.user_text'}}"
{{/each}}
""")

user_agent = user_agent(conversation=[{"user_text": "Hello, how are you?"}])
for i in range(10):
    bot_agent = bot_agent(user_text=user_agent["conversation"][-2]["user_text"])
    user_agent = user_agent(bot_text=bot_agent["conversation"][-2]["bot_text"])

# dump the whole conversation
user_agent["conversation"]

[{'user_text': 'Hello, how are you?'},
 {'bot_text': "I'm good, thank you for asking. How about you?",
  'user_text': "I'm good, too. How are you today?"},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not real